In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from scipy.stats import chi2_contingency
from collections import Counter, OrderedDict
from sklearn.pipeline import Pipeline
import statsmodels.api as sm  # <--- Yay! API! 
from pandas.tseries.frequencies import to_offset
from sklearn.pipeline import make_pipeline
from scipy.stats import chi2_contingency
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import roc_auc_score
from numpy.linalg import inv
import category_encoders
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_curve
from sklearn.tree import DecisionTreeClassifier
from statsmodels.tsa.seasonal import seasonal_decompose
import itertools
from pandas.plotting import lag_plot
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
# NLTK imports
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import pickle

In [2]:
item_pair_train = pd.read_csv('ItemPairs_train.csv')
item_pair_test = pd.read_csv('ItemPairs_test.csv')
item_info_train = pd.read_csv('ItemInfo_train.csv')
item_info_test = pd.read_csv('ItemInfo_test.csv')
category = pd.read_csv('Category.csv')
location = pd.read_csv('Location.csv')

In [4]:
print(item_pair_train.shape)
print(item_pair_test.shape)
print(item_info_train.shape)
print(item_info_test.shape)
print(category.shape)
print(location.shape)

(2991396, 4)
(1044196, 3)
(3344613, 11)
(1315205, 11)
(51, 2)
(3449, 2)


In [5]:
item_pair_train.head(3)

,itemID_1,itemID_2,isDuplicate,generationMethod
0,1,4112648,1,1
1,3,1991275,1,1
2,4,1223296,0,1


In [6]:
item_info_train.head(10)

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon
0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924
1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,NaN,55.678037,37.256548
2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458
3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194
4,8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,NaN,55.777170,37.586194
5,9,39,Сноуборд ботинки Nitro Team 10 us,"сноубордические ботинки Nitro Team\nразмер 42,...","12418395, 9930491","{""Вид товара"":""Зимние виды спорта""}",7000.0,644200,NaN,58.004785,56.237654
6,12,9,"LADA Priora, 2015",Машина новая пробег реальный. Не битая не краш...,"1338189, 1648456, 6321889, 9883716","{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",445000.0,631060,NaN,44.219841,42.058825
7,15,32,Телевизор,"Телевизоры кинескопные, диагональ от 37 см. до...","11244051, 14467554, 2240467, 5099565, 8002433","{""Вид товара"":""Телевизоры и проекторы""}",1600.0,662810,NaN,57.622434,39.887894
8,16,27,Шуба мутоновая,"качественная, производство Казань, хорошее сос...","11762574, 316289, 4015142","{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",1000.0,657600,NaN,56.495116,84.972128
9,19,88,Массажная накидка Beurer mg,Распродаем остатки. Все модели хиты продаж!!! ...,9722988,"{""Вид товара"":""Приборы и аксессуары""}",5000.0,637640,500701.0,55.640538,37.606065


In [7]:
item_pair_test.head(3)

,id,itemID_1,itemID_2
0,0,5,4670875
1,1,5,787210
2,2,6,1705280


In [8]:
category.head(3)

,categoryID,parentCategoryID
0,9,1
1,10,1
2,11,1


In [9]:
## How many different products we have
products = item_info_train['itemID'].unique()
products.shape

(3344613,)

In [10]:
## How many categories and their balances
category_list = category['categoryID']
print(category_list.shape)
item_info_train['categoryID'].value_counts()

(51,)


10     364758
27     364403
9      264839
84     253259
29     192378
112    169631
24     140271
36     137336
20     120283
19     115359
40      99949
39      89328
30      88845
101     78939
114     77820
21      75682
32      60641
97      42881
25      37215
83      36668
90      34356
89      33891
96      33259
81      32869
98      30042
28      28973
88      27066
23      24322
105     24042
33      21624
26      21609
38      21452
34      21130
31      19066
115     18983
87      18323
102     17941
42      14748
82      11946
99      11301
14      11078
93       9727
106      9241
85       9212
111      5984
94       5983
92       5509
91       4260
11       2341
116      2060
86       1790
Name: categoryID, dtype: int64

In [11]:
## Check the price distribution for each category
price_cat = item_info_train[['categoryID', 'price']].groupby('categoryID').median()
price_cat

,price
categoryID,
9,284000.0
10,4000.0
11,55000.0
14,36500.0
19,2400.0
20,3500.0
21,4500.0
23,12000.0
24,1200000.0


In [12]:
json_attributes = np.array(item_info_train['attrsJSON'])

In [13]:
list_of_dict_on_attributes = []
for i in json_attributes:
    if isinstance(i, str):
        list_of_dict_on_attributes.append(json.loads(i))

In [14]:
list_of_keys = []
for i in list_of_dict_on_attributes:
    key_list=list(i.keys())
    for j in key_list:
        list_of_keys.append(j)

In [15]:
count_expressions = Counter(list_of_keys)

In [16]:
total = 0
for i in count_expressions.keys():
    total = total + count_expressions[i]

In [17]:
total

13521265

In [18]:
l = count_expressions.most_common(70)
s = 0
for i in l:
    s = s + i[1]
print(s/total)
# We have 13521265. To account for 91% of those counts we would need to translate 70 expressions/words.
# Not convenient

0.9144252405377751


In [19]:
# Вид товара: Type of product
# Вид одежды: Kind of clothes
# Предмет одежды: Piece of clothing
# Размер: Size
# Тип товара: Product type:
# Коробка передач: Transmission
# Тип кузова: Body type
# Тип двигателя: Type of Engine
# Тип автомобиля: Type of Car
# Марка: Model

# Зимние виды спорта: Winter sports
# Год выпуска:Year of issue
# Количество дверей: Number of doors
# Марка: Brand
# Модель: Model
# Мощность двигателя: Engine power
# Объём двигателя: Engine capacity

In [20]:
## Consider all the words in the title and attrJSON fields

In [ ]:
tokenizer = WordPunctTokenizer()
stemmer = SnowballStemmer("russian", ignore_stopwords=True)

In [ ]:
X_title = item_info_train['title'].copy()
X_ajson = item_info_train['attrsJSON'].copy()
total_docs = pd.DataFrame(pd.concat([X_title, X_ajson], ignore_index=True))
total_docs.columns = ['text']
total_docs.fillna('N/A', inplace=True)
total_docs.head(3)
total_docs.index.nunique()

In [ ]:
X_title.shape[0] + X_ajson.shape[0]

In [ ]:
#First step clean the texts as in BLU 7. The final objective: built the vocabulary
def clean(doc):
    # remove html tags
    doc = re.sub("[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)","", doc)
    doc = re.sub(r"\W", " ", doc)  
    # lowercase
    doc = doc.lower()
    # tokenize
    words = tokenizer.tokenize(doc)
    # remove punctuation
    words = list(filter(lambda x: x not in string.punctuation, words))
    # stem
    stems = list(map(stemmer.stem, words))
    new_doc = " ".join(stems)
    return new_doc

In [ ]:
## Pay attention fill na
total_docs_cleaned = total_docs['text'].apply(clean)
total_docs_cleaned.to_csv('cleaned_docs.csv')

In [4]:
total_docs_cleaned = pd.read_csv('cleaned_docs.csv', header=None)

In [5]:
total_docs_cleaned.columns = ['index', 'text']
total_docs_cleaned.drop(columns=['index'], inplace=True)
total_docs_cleaned.head()

,text
0,прод камаз
1,yamah r
2,iphon gs gb
3,xiaom mi гб ram гб rom бел
4,лыжн ботинк


In [24]:
def build_vocabulary(docs):
    vocabulary = Counter()
    for index, row in docs.iterrows():
        doc = str(row['text'])
        words = doc.split()
        vocabulary.update(words)
    
    return OrderedDict(vocabulary.most_common())

In [3]:
vocabulary = build_vocabulary(total_docs_cleaned)

NameError: name 'build_vocabulary' is not defined

In [ ]:
with open( 'vocabulary.pickle', 'wb' ) as output_file:
    pickle.dump(vocabulary, output_file)

In [3]:
with open( 'vocabulary.pickle', 'rb' ) as input_file:
    vocabulary = pickle.load(input_file)

In [7]:
def vectorize(vocabulary, docs):
    vectors = []
    for index, row in docs.iterrows():
        doc = str(row['text'])
        words = doc.split()
        vector = np.array([doc.count(word) for word in vocabulary])
        vectors.append(vector)
        #for doc in docs:
        #    words = doc.split()
        #    vector = np.array([doc.count(word) for word in vocabulary])
        #    vectors.append(vector)

    return vectors

In [ ]:
vecs = vectorize(vocabulary, total_docs_cleaned)
with open( 'vectors.pickle', 'wb' ) as output_file:
    pickle.dump(vecs, output_file)